# Ellipszoidi földrajzi koordináták számítása
---

### Csomagok importálása

In [3]:
from math import *
import numpy as np

import utils

### Kiinduló adatok

A 2. számú pont GRS80 ellipszoidra vonatkozó ellipszoidi földrajzi koordinátái:  

$$ \varphi =  45 - 52 - 59,3690 $$ 
$$ \lambda =  17 - 46 - 38,7912 $$
$$ h = 145,654 m $$

In [4]:
fi_dms = (45, 52, 59.3690)
lambda_dms = (17, 46, 38.7912)
h = 145.654

A GRS80 ellipszoid paraméterei:
$$ a = 6 378 137,000 m $$
$$ e^2 = 0,006 694 380 0229 $$

In [5]:
a = 6378137
e2 = 0.0066943800229

Szögek átváltása fokba:

In [6]:
fi = utils.dms_to_deg(fi_dms)
lambda_ = utils.dms_to_deg(lambda_dms)

print('fi = {}°'.format(fi))
print('lambda = {}°'.format(lambda_))

fi = 45.883158055555555°
lambda = 17.777442°


Szögek átváltása radiánba:

In [7]:
fi_rad = np.deg2rad(fi)
lambda_rad = np.deg2rad(lambda_)

print('fi = {} rad'.format(fi_rad))
print('lambda = {} rad'.format(lambda_rad))

fi = 0.8008121792824038 rad
lambda = 0.3102748954823258 rad


### Harántgörbületi sugár

$$ N = \frac{a}{(1 - e^2 sin^2(\varphi))^{1/2}} $$

In [8]:
N = a / pow((1 - e2 * pow(sin(fi_rad), 2)), 1 / 2)
print('N = {} m'.format(round(N, 3)))

N = 6389168.994 m


### Meridián irányú görbületi sugár

$$ M = \frac{a (1 - e^2)}{(1 - e^2 sin^2\varphi)^{3/2}} $$

In [9]:
M = (a * (1 - e2)) / pow((1 - e2 * pow(sin(fi_rad), 2)), 3 / 2)

print('M = {} m'.format(round(M, 3)))

M = 6368370.646 m


### Gauss-féle középgörbületi sugár

$$ R = \sqrt{MN} $$

In [10]:
R = sqrt(M * N)

print('R = {} m'.format(round(R, 3)))

R = 6378761.343 m


### Helyvektorok

A P pont P '' ellipszoidi megfelelőjének helyvektora:
$$
\vec{r_{p''}} = N
\begin{vmatrix}
cos \varphi cos \lambda \\
cos \varphi sin \lambda \\
sin \varphi \\
\end{vmatrix}
$$

A P pont helyvektora:
$$
\vec{r_p} =
\begin{vmatrix}
(N + h)cos \varphi cos \lambda \\
(N + h)cos \varphi sin \lambda \\
((1 - e^2)N + h)sin \varphi \\
\end{vmatrix} = 
\begin{vmatrix}
x \\
y \\
z \\
\end{vmatrix}
$$

In [11]:
x = (N + h) * cos(fi_rad) * cos(lambda_rad)
y = (N + h) * cos(fi_rad) * sin(lambda_rad)
z = ((1 - e2) * N + h) * sin(fi_rad)

print('x = {} m'.format(round(x, 3)))
print('y = {} m'.format(round(y, 3)))
print('z = {} m'.format(round(z, 3)))

x = 4235372.644 m
y = 1357990.217 m
z = 4556321.059 m


### Geocentrikus szélesség

$$ \psi = arctg(\frac{z}{\sqrt{x^2 + y^2}}) $$

In [12]:
psi_rad = atan(z / (sqrt(pow(x, 2) + pow(y, 2))))

psi = np.rad2deg(psi_rad)

psi_dms = utils.deg_to_dms(psi)

print('psi = {} rad'.format(round(psi_rad, 3)))
print('psi = {}°'.format(round(psi, 3)))
print('psi = {}-{}-{}'.format(psi_dms[0], psi_dms[1], round(psi_dms[2], 4)))

psi = 0.797 rad
psi = 45.691°
psi = 45-41-26.9187


### Geocentrikus távolság

$$ r = \sqrt{x^2 + y^2 + z^2} $$

In [13]:
r = sqrt(pow(x, 2) + pow(y, 2) + pow(z, 2))

print('r = {} m'.format(round(r, 3)))

r = 6367305.589 m


## Ellenőrzés Bowring közelítő eljárással

### Ellipszoid fél kistengely hossza

$$ b = a\sqrt{1 - e^2} $$

In [14]:
b = a * sqrt(1 - e2)

print('b = {} m'.format(round(b, 3)))

b = 6356752.314 m


### Második numerikus excentricitás négyzete

$$ e'^2 = \frac{e^2}{1 - e^2} = \frac{a^2 - b^2}{b^2} $$

In [15]:
# ex2 = e2 / (1 - e2)
ex2 = (pow(a, 2) - pow(b, 2)) / pow(b, 2)

print('ex2 = {}'.format(ex2))

ex2 = 0.006739496775478272


###  P pont Z tengelytől mért távolsága

In [16]:
p = sqrt(pow(x, 2) + pow(y, 2))

print('p = {} m'.format(round(p, 3)))

p = 4447754.362 m


### Segédszög

$$ \Theta = arctan\frac{z * a}{p * b} $$

In [17]:
theta_rad = atan((z * a) / (p * b))

theta_deg = np.rad2deg(theta_rad)

theta_dms = utils.deg_to_dms(theta_deg)

print('theta = {}-{}-{}'.format(theta_dms[0], theta_dms[1], round(theta_dms[2], 4)))

theta = 45-47-13.1677


### Ellipszoidi szélessség

$$ \varphi = arctan \frac{z + e'^2 b sin^3\Theta}{p - e^2 a cos^3\Theta} $$

In [18]:
fi_ell_rad = atan((z + ex2 * b * pow(sin(theta_rad), 3)) / (p - e2 * a * pow(cos(theta_rad), 3)))

fi_ell_deg = np.rad2deg(fi_ell_rad)

fi_ell_dms = utils.deg_to_dms(fi_ell_deg)

# Listává alakítjuk. kerekítjük az utolsó számot (mp) és visszaalakítjuk tuple-re.
fi_ell_dms = list(fi_ell_dms)
fi_ell_dms[2] = round(fi_ell_dms[2], 4)
fi_ell_dms = tuple(fi_ell_dms)

print('fi_ell = {}-{}-{}'.format(fi_ell_dms[0], fi_ell_dms[1], fi_ell_dms[2]))

fi_ell = 45-52-59.369


In [19]:
if fi_ell_dms == fi_dms:
    print('✔ Az ellipszoidi szélességek megegyeznek!')
else:
    print('❌ Az ellipszoidi szélesség nem egyeznek meg!')

✔ Az ellipszoidi szélességek megegyeznek!


### Ellipszoidi hosszúság

$$ \lambda = arctan\frac{y}{x} $$

In [20]:
lambda_ell_rad = atan(y / x)

lambda_ell_deg = np.rad2deg(lambda_ell_rad)

lambda_ell_dms = utils.deg_to_dms(lambda_ell_deg)

# Listává alakítjuk. kerekítjük az utolsó számot (mp) és visszaalakítjuk tuple-re.
lambda_ell_dms = list(lambda_ell_dms)
lambda_ell_dms[2] = round(lambda_ell_dms[2], 4)
lambda_ell_dms = tuple(lambda_ell_dms)

print('fi_ell = {}-{}-{}'.format(lambda_ell_dms[0], lambda_ell_dms[1], lambda_ell_dms[2]))

fi_ell = 17-46-38.7912


In [21]:
if lambda_ell_dms == lambda_dms:
    print('✔ Az ellipszoidi hosszúságok megegyeznek!')
else:
    print('❌ Az ellipszoidi hosszúságok nem egyeznek meg!')

✔ Az ellipszoidi hosszúságok megegyeznek!


### Ellipszoid feletti magasság

$$ h = \frac{p}{cos\varphi} - N $$

In [22]:
h_ell = p / cos(fi_ell_rad) - N

h_ell = round(h_ell, 3)

print('h_ell = {} m'.format(h_ell))

h_ell = 145.654 m


In [23]:
if h_ell == h_ell:
    print('✔ Az ellipszoid feletti magasságok megegyeznek!')
else:
    print('❌ Az ellipszoid feletti magasságok nem egyeznek meg!')

✔ Az ellipszoid feletti magasságok megegyeznek!


### Ellenőrzés összegzése

In [24]:
if fi_ell_dms == fi_dms:
    print('✔ Az ellipszoidi szélességek megegyeznek!')
else:
    print('❌ Az ellipszoidi szélesség nem egyeznek meg!')
    
if lambda_ell_dms == lambda_dms:
    print('✔ Az ellipszoidi hosszúságok megegyeznek!')
else:
    print('❌ Az ellipszoidi hosszúságok nem egyeznek meg!')

if h_ell == h_ell:
    print('✔ Az ellipszoid feletti magasságok megegyeznek!')
else:
    print('❌ Az ellipszoid feletti magasságok nem egyeznek meg!')

✔ Az ellipszoidi szélességek megegyeznek!
✔ Az ellipszoidi hosszúságok megegyeznek!
✔ Az ellipszoid feletti magasságok megegyeznek!


### Ellenőrzés az egyetemi weboldalon 

In [35]:
from urllib import request, parse

from IPython.display import HTML

# Post paraméterek enkódolása bájtokká.
data = parse.urlencode({'neptun': 'WJ81S1', 'rgeoc': round(r, 3), 'pszi': '{}-{}-{}'.format(psi_dms[0], psi_dms[1], round(psi_dms[2], 4)), 'rgauss': round(R, 3)}).encode()
# Request létrehozása.
req = request.Request('http://www.agt.bme.hu/gtoth/fg/process1hf.php', data=data)
# Request megnyitása. 
resp = request.urlopen(req)
# Response kiolvasása, dekódolása, majd megjelenítése.
resp_str = resp.read().decode('utf-8')

# Workaround: a GitHub IPythonja nem tudja lerenderelni a font tag-eket.
resp_str = resp_str.replace('<font', '<span').replace('\font', '\span')

HTML(resp_str)